In [1]:
# Vizuelna podešavanja okruženja (samo razvuče notebook na širinu ekrana)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [2]:
import networkx as nx
import pickle

with open('data_cleaned/data2020_cleaned', 'rb') as file:
    data = pickle.load(file)
print(data)

                            tourney_id                  tourney_name surface  \
0                            2020-8888                       Atp Cup    Hard   
1                            2020-8888                       Atp Cup    Hard   
2                            2020-8888                       Atp Cup    Hard   
3                            2020-8888                       Atp Cup    Hard   
4                            2020-8888                       Atp Cup    Hard   
...                                ...                           ...     ...   
1442  2020-M-DC-2020-WG2-PO-POL-HKG-01  Davis Cup WG2 PO: POL vs HKG    Hard   
1443  2020-M-DC-2020-WG2-PO-POL-HKG-01  Davis Cup WG2 PO: POL vs HKG    Hard   
1444  2020-M-DC-2020-WG2-PO-SYR-ZIM-01  Davis Cup WG2 PO: SYR vs ZIM    Hard   
1445  2020-M-DC-2020-WG2-PO-SYR-ZIM-01  Davis Cup WG2 PO: SYR vs ZIM    Hard   
1446  2020-M-DC-2020-WG2-PO-SYR-ZIM-01  Davis Cup WG2 PO: SYR vs ZIM    Hard   

      draw_size tourney_level  tourney_

In [3]:
lista_winners = set(data['winner_id'].unique())
lista_losers = set(data['loser_id'].unique())

svi_igraci = lista_winners.union(lista_losers)
print(f"Ukupan broj igraca je {len(svi_igraci)}, a ukupan broj meceva je {data.shape[0]}")


Ukupan broj igraca je 345, a ukupan broj meceva je 1447


In [4]:
import pandas as pd

dataPath = 'data/atp_players.csv'
players = pd.read_csv(dataPath)

dataPath = 'data/atp_rankings_10s.csv'
rankings = pd.read_csv(dataPath)

In [5]:
G = nx.Graph()
for player_id in svi_igraci:
    player = players[players['player_id'] == player_id]
    if player.empty:
        print("Postoji id igraca koji ne postoji u listi svih igraca!")
    else:
        player_rank = rankings[(rankings['player'] == player_id) & (rankings['ranking_date'] > 20200000) & (rankings['ranking_date'] < 20210000)]
        rank_p = 0
        points_p = 0
        
        if not player_rank.empty:
            last_row = player_rank.shape[0] - 1
            rank_p = int(player_rank.iloc[last_row, 1])
            points_p = int(player_rank.iloc[last_row, 3])
            
        G.add_node(str(player_id), first_name = str(player.iloc[0, 1]), last_name = str(player.iloc[0, 2]), country = str(player.iloc[0, 5]), hand = str(player.iloc[0, 3]), rank = rank_p, points = points_p)


In [6]:
data_reduced = data[['winner_id', 'loser_id']]
data_reduced = data_reduced.astype({'winner_id': str, 'loser_id': str})

for _, winner_id, loser_id in data_reduced.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1)

In [7]:
output_path = "models/model2020.gml"
nx.write_gml(G, output_path)

In [8]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 345
Number of edges: 1325
Average degree:   7.6812
